In [1]:
import pandas as pd
from api.core.mongo import db
from langchain_text_splitters import RecursiveJsonSplitter
from nomic import embed
import logging

logging.basicConfig(level=logging.DEBUG)

In [2]:
rows = pd.DataFrame(list(db["products"].find({
    "product_type": {"$in": ["ski", "phone"]},
}))).drop(columns=["_id"]).to_dict(orient="records")

DEBUG:pymongo.serverSelection:{"message": "Server selection started", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 66f98404081255c9e70a576c, topology_type: ReplicaSetWithPrimary, servers: [<ServerDescription ('cluster0-shard-00-00.6le3g.mongodb.net', 27017) server_type: RSSecondary, rtt: 0.06199999999989814>, <ServerDescription ('cluster0-shard-00-01.6le3g.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('cluster0-shard-00-02.6le3g.mongodb.net', 27017) server_type: RSPrimary, rtt: 0.06199999999989814>]>", "clientId": {"$oid": "66f98404081255c9e70a576c"}}
DEBUG:pymongo.serverSelection:{"message": "Server selection succeeded", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 66f98404081255c9e70a576c, topology_type: ReplicaSetWithPrimary, servers: [<ServerDescription ('cluster0-shard-00-00.6le3g.mongodb.net', 27017) server_type: RSSecondary, rtt: 0.06199999999989814>, 

In [3]:
splitter = RecursiveJsonSplitter(max_chunk_size=300)
documents = splitter.create_documents(rows)

In [ ]:
# Store the data as vector embeddings in Atlas

#: create groups of 100 documents
for i in range(0, len(documents), 100):
    embeddings = embed.text([document.page_content for document in documents[i:i+100]], inference_mode="local")
    db["embeddings"].insert_many([{
        "embedding": embedding
    } for embedding in embeddings["embeddings"]])
    print(f"Inserted {i+100} documents")
        